<a href="https://colab.research.google.com/github/tlsdmswn01/Text_mining_project/blob/main/%EB%89%B4%EC%8A%A4%EA%B8%B0%EC%82%AC%20%ED%81%AC%EB%A1%A4%EB%A7%81/%EC%A2%85%EB%AA%A9%EB%A6%AC%ED%8F%AC%ED%8A%B8_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 종목리포트 크롤링

import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

text_list = []
result = []

for i in range(1,2):
    url = 'http://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={}&reportId=0&searchKey=stock&searchValue=삼성전자'.format(i)

    res = requests.get(url)

    html = BeautifulSoup(res.text, 'html.parser')
    html = html.find('ul', class_='board-list pnesec100 pnesec')

    for j in range(1, 31):
        for k in range(6):
            text = html.find_all('li')[j].find_all('div')[k].text
            text_list.append(re.sub(r'\n|\t|\r', '',text))

    for i in range(0, len(text_list), 6):
        result.append(text_list[i:i+6])

columns = ['종목', '평가', '적정가격', '투자의견', '증권사', '날짜']
df = pd.DataFrame(result, columns=columns)

df['종목'] = df['종목'].apply(lambda x: x.replace('[', '').replace(']', ''))
df['적정가격'] = df['적정가격'].apply(lambda x: x.replace('적정가격', '').replace('원', '').replace(" ", ''))
df['날짜'] = pd.to_datetime(df['날짜'])
df = df[df['날짜'] >= '2023-01-01']
price_df = df[df['투자의견'] != '-']
price_df['적정가격'] = price_df['적정가격'].astype('int')

In [ ]:
df

### 적정 가격 시각화

In [ ]:
# 폰트 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt

In [ ]:
# 그래프 그리기
plt.rc('font', family='NanumBarunGothic')
plt.figure(figsize=(10, 6))
plt.plot(price_df['날짜'], price_df['적정가격'], marker='o', linestyle='--')
plt.title('날짜별 적정가격 그래프', fontsize=16)
plt.xticks(price_df['날짜'], [f'{date.strftime("%Y-%m-%d")} ({broker})' for date, broker in zip(price_df['날짜'], price_df['증권사'])], rotation=45)
plt.tight_layout()
plt.show()